# Food Price Data Source

[WFP Food Prices Kenya Dataset](https://data.humdata.org/dataset/e0d3fba6-f9a2-45d7-b949-140c455197ff/resource/517ee1bf-2437-4f8c-aa1b-cb9925b9d437/download/wfp_food_prices_ken.csv)

# -------------------------------------------------------------------------------------------






In [3]:
!pip install pyspark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions  import col,to_date,regexp_replace

spark = SparkSession.builder.appName('FoodPricePrediction').master('local[*]').getOrCreate()

spark.sparkContext.appName

'FoodPricePrediction'

In [6]:
food_prices = spark.read.csv("drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets/raw_data/wfp_food_prices_ken.csv",inferSchema=True,header=True)
food_prices.printSchema()

root
 |-- date: string (nullable = true)
 |-- admin1: string (nullable = true)
 |-- admin2: string (nullable = true)
 |-- market: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- category: string (nullable = true)
 |-- commodity: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- priceflag: string (nullable = true)
 |-- pricetype: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- price: string (nullable = true)
 |-- usdprice: string (nullable = true)



In [7]:
food_prices.count()

12865

In [8]:
food_prices.show(5)

+----------+----------+----------+----------------+--------+---------+------------------+-------------+----------+----------------+----------------+---------+------+----------+
|      date|    admin1|    admin2|          market|latitude|longitude|          category|    commodity|      unit|       priceflag|       pricetype| currency| price|  usdprice|
+----------+----------+----------+----------------+--------+---------+------------------+-------------+----------+----------------+----------------+---------+------+----------+
|     #date|#adm1+name|#adm2+name|#loc+market+name|#geo+lat| #geo+lon|        #item+type|   #item+name|#item+unit|#item+price+flag|#item+price+type|#currency|#value|#value+usd|
|2006-01-15|     Coast|   Mombasa|         Mombasa|   -4.05|39.666667|cereals and tubers|        Maize|        KG|          actual|       Wholesale|      KES| 16.13|    0.2235|
|2006-01-15|     Coast|   Mombasa|         Mombasa|   -4.05|39.666667|cereals and tubers|Maize (white)|     90 KG| 

## Filter rows where any column is NULL or an empty string

In [9]:
missing_rows = food_prices.filter(
    (col("date").isNull()) | (col("date") == "") |
    (col("admin1").isNull()) | (col("admin1") == "") |
    (col("admin2").isNull()) | (col("admin2") == "") |
    (col("price").isNull()) | (col("price") == "") |
    (col("usdprice").isNull()) | (col("usdprice") == "")
)

missing_rows.show(truncate=False)

+----------+------+------+-----------------+--------+---------+------------------+-------------+----+---------+---------+--------+-----+--------+
|date      |admin1|admin2|market           |latitude|longitude|category          |commodity    |unit|priceflag|pricetype|currency|price|usdprice|
+----------+------+------+-----------------+--------+---------+------------------+-------------+----+---------+---------+--------+-----+--------+
|2015-05-15|NULL  |NULL  |Hola (Tana River)|NULL    |NULL     |cereals and tubers|Maize (white)|KG  |actual   |Retail   |KES     |38.0 |0.3967  |
|2015-06-15|NULL  |NULL  |Hola (Tana River)|NULL    |NULL     |cereals and tubers|Maize (white)|KG  |actual   |Retail   |KES     |36.0 |0.3708  |
|2015-07-15|NULL  |NULL  |Hola (Tana River)|NULL    |NULL     |cereals and tubers|Maize (white)|KG  |actual   |Retail   |KES     |42.0 |0.4126  |
|2015-08-15|NULL  |NULL  |Hola (Tana River)|NULL    |NULL     |cereals and tubers|Maize (white)|KG  |actual   |Retail   |KES

In [10]:
#Drop NuLL values

food_data = food_prices.dropna()

In [11]:
food_data.show(5)

+----------+----------+----------+----------------+--------+---------+------------------+-------------+----------+----------------+----------------+---------+------+----------+
|      date|    admin1|    admin2|          market|latitude|longitude|          category|    commodity|      unit|       priceflag|       pricetype| currency| price|  usdprice|
+----------+----------+----------+----------------+--------+---------+------------------+-------------+----------+----------------+----------------+---------+------+----------+
|     #date|#adm1+name|#adm2+name|#loc+market+name|#geo+lat| #geo+lon|        #item+type|   #item+name|#item+unit|#item+price+flag|#item+price+type|#currency|#value|#value+usd|
|2006-01-15|     Coast|   Mombasa|         Mombasa|   -4.05|39.666667|cereals and tubers|        Maize|        KG|          actual|       Wholesale|      KES| 16.13|    0.2235|
|2006-01-15|     Coast|   Mombasa|         Mombasa|   -4.05|39.666667|cereals and tubers|Maize (white)|     90 KG| 

In [12]:
food_data.count()

12825

In [13]:
food_prices_data = (food_data
    .filter(col('date') != '#date')  # Remove header row in data
    .withColumnRenamed('date', 'date')
    .withColumnRenamed('admin1', 'region')
    .withColumnRenamed('admin2', 'county')
    .withColumnRenamed('market', 'market_name')
    .withColumnRenamed('latitude', 'lat')
    .withColumnRenamed('longitude', 'lon')
    .withColumnRenamed('category', 'food_category')
    .withColumnRenamed('commodity', 'food_item')
    .withColumnRenamed('unit', 'measurement_unit')
    .withColumnRenamed('priceflag', 'price_flag')
    .withColumnRenamed('pricetype', 'price_type')
    .withColumnRenamed('currency', 'currency')
    .withColumnRenamed('price', 'local_price')
    .withColumnRenamed('usdprice', 'usd_price')
    .withColumn('date', to_date(col('date'), 'yyyy-MM-dd'))
    .withColumn('local_price', col('local_price').cast('float'))
    .withColumn('usd_price', col('usd_price').cast('float'))

)

In [59]:
# Dropping unnecessary columns

food_prices1 = food_prices_data.drop('lat','lon','price_flag', 'price_type', 'currency', 'usd_price')

In [60]:
food_prices1.show(3)

+----------+------+-------+-----------+------------------+-------------+----------------+-----------+
|      date|region| county|market_name|     food_category|    food_item|measurement_unit|local_price|
+----------+------+-------+-----------+------------------+-------------+----------------+-----------+
|2006-01-15| Coast|Mombasa|    Mombasa|cereals and tubers|        Maize|              KG|      16.13|
|2006-01-15| Coast|Mombasa|    Mombasa|cereals and tubers|Maize (white)|           90 KG|     1480.0|
|2006-01-15| Coast|Mombasa|    Mombasa|   pulses and nuts|        Beans|              KG|      33.63|
+----------+------+-------+-----------+------------------+-------------+----------------+-----------+
only showing top 3 rows



In [61]:
food_prices1.select("measurement_unit").distinct().show(truncate=False)

+----------------+
|measurement_unit|
+----------------+
|Bunch           |
|400 G           |
|64 KG           |
|Head            |
|L               |
|200 G           |
|50 KG           |
|13 KG           |
|90 KG           |
|126 KG          |
|KG              |
|Unit            |
|500 ML          |
+----------------+



In [62]:
from pyspark.sql.functions import when, trim, regexp_extract, col

food_prices2 = food_prices1.withColumn(
    "normalized_unit",
    when(trim(col("measurement_unit")) == "400 G", 0.4)
    .when(trim(col("measurement_unit")) == "200 G", 0.2)
    .when(trim(col("measurement_unit")) == "500 ML", 0.5)  # Assuming it's a liter equivalent
    .when(trim(col("measurement_unit")) == "L", 1.0)
    .when(trim(col("measurement_unit")) == "KG", 1.0)  # Treat single "KG" as 1.0
    .when(trim(col("measurement_unit")).isin(["Bunch", "Head", "Unit"]), -1)  # Placeholder for non-numeric units
    .when(trim(col("measurement_unit")).endswith("KG"),
          regexp_extract(trim(col("measurement_unit")), r'(\d+)', 1).cast("float"))  # Extract number before "KG"
    .otherwise(None)  # Catch any unexpected cases
)

food_prices2.show(10)

+----------+-------+--------+-----------+------------------+----------------+----------------+-----------+---------------+
|      date| region|  county|market_name|     food_category|       food_item|measurement_unit|local_price|normalized_unit|
+----------+-------+--------+-----------+------------------+----------------+----------------+-----------+---------------+
|2006-01-15|  Coast| Mombasa|    Mombasa|cereals and tubers|           Maize|              KG|      16.13|            1.0|
|2006-01-15|  Coast| Mombasa|    Mombasa|cereals and tubers|   Maize (white)|           90 KG|     1480.0|           90.0|
|2006-01-15|  Coast| Mombasa|    Mombasa|   pulses and nuts|           Beans|              KG|      33.63|            1.0|
|2006-01-15|  Coast| Mombasa|    Mombasa|   pulses and nuts|     Beans (dry)|           90 KG|     3246.0|           90.0|
|2006-01-15|Eastern|   Kitui|      Kitui|cereals and tubers|   Maize (white)|              KG|       17.0|            1.0|
|2006-01-15|East

In [63]:
from pyspark.sql.functions import max,min

food_prices2.select(min('date')).show()
food_prices2.select(max('date')).show()

+----------+
| min(date)|
+----------+
|2006-01-15|
+----------+

+----------+
| max(date)|
+----------+
|2024-12-15|
+----------+



In [64]:
from pyspark.sql.functions import year, month, date_format

# Extract year and month, then drop the date column
food_prices2 = food_prices2.withColumn('year', year('date')).withColumn("month", date_format("date", "MMM")).drop('date')


In [66]:
# Show the updated table
food_prices2.show(50)

+-------------+-----------+--------------------+------------------+--------------------+----------------+-----------+---------------+----+-----+
|       region|     county|         market_name|     food_category|           food_item|measurement_unit|local_price|normalized_unit|year|month|
+-------------+-----------+--------------------+------------------+--------------------+----------------+-----------+---------------+----+-----+
|        Coast|    Mombasa|             Mombasa|cereals and tubers|               Maize|              KG|      16.13|            1.0|2006|  Jan|
|        Coast|    Mombasa|             Mombasa|cereals and tubers|       Maize (white)|           90 KG|     1480.0|           90.0|2006|  Jan|
|        Coast|    Mombasa|             Mombasa|   pulses and nuts|               Beans|              KG|      33.63|            1.0|2006|  Jan|
|        Coast|    Mombasa|             Mombasa|   pulses and nuts|         Beans (dry)|           90 KG|     3246.0|           90

# -------------------------------------------------------------------------------------------

# Climate Data Data Source

[Climate Kenya Dataset](https://open.africa/dataset/kenya-climate-data-1991-2016)

In [78]:
rainfall_data = spark.read.csv("drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets/raw_data/kenya-climate-data-1991-2016-rainfallmm.csv",inferSchema=True,header=True)
rainfall_data.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month Average: string (nullable = true)
 |-- Rainfall - (MM): double (nullable = true)



In [79]:
rainfall_data.show(5)

+----+-------------+---------------+
|Year|Month Average|Rainfall - (MM)|
+----+-------------+---------------+
|1991|  Jan Average|        38.2847|
|1991|  Feb Average|        12.7492|
|1991|  Mar Average|        73.3656|
|1991|  Apr Average|         83.135|
|1991|  May Average|        112.275|
+----+-------------+---------------+
only showing top 5 rows



In [80]:
temperature_data = spark.read.csv("drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets/raw_data/kenya-climate-data-1991-2016-temp-degress-celcius.csv",inferSchema=True,header=True)
temperature_data.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month Average: string (nullable = true)
 |-- Temperature - (Celsius): double (nullable = true)



In [81]:
temperature_data.show(5)

+----+-------------+-----------------------+
|Year|Month Average|Temperature - (Celsius)|
+----+-------------+-----------------------+
|1991|  Jan Average|                25.1631|
|1991|  Feb Average|                26.0839|
|1991|  Mar Average|                26.2236|
|1991|  Apr Average|                25.5812|
|1991|  May Average|                24.6618|
+----+-------------+-----------------------+
only showing top 5 rows



In [82]:
# Combining rainfall and temperature data into one dataset
climate_data = (rainfall_data
    .withColumnRenamed('Year', 'year')
    .withColumnRenamed('Month Average', 'month')
    .withColumnRenamed('Rainfall - (MM)', 'rainfall_mm')
    .withColumn('rainfall_mm', col('rainfall_mm').cast('float'))
    .join(temperature_data
          .withColumnRenamed('Year', 'year')
          .withColumnRenamed('Month Average', 'month')
          .withColumnRenamed('Temperature - (Celsius)', 'temperature_c')
          .withColumn('temperature_c', col('temperature_c').cast('float')),
          on=['year', 'month'], how='inner')
    .withColumn('month', regexp_replace('month', ' Average', ''))  # Remove 'Average' from month names
    .dropna(subset=['year', 'month'])
)

In [83]:
climate_data.show(5)

+----+-----+-----------+-------------+
|year|month|rainfall_mm|temperature_c|
+----+-----+-----------+-------------+
|1991|  Jan|    38.2847|      25.1631|
|1991|  Feb|    12.7492|      26.0839|
|1991|  Mar|    73.3656|      26.2236|
|1991|  Apr|     83.135|      25.5812|
|1991|  May|    112.275|      24.6618|
+----+-----+-----------+-------------+
only showing top 5 rows



In [84]:
from pyspark.sql.functions import max,min

climate_data.select(min('year')).show()
climate_data.select(max('year')).show()

+---------+
|min(year)|
+---------+
|     1991|
+---------+

+---------+
|max(year)|
+---------+
|     2016|
+---------+



In [85]:
from pyspark.sql.functions import col, mean

# Filter data to keep only records from 2006 onwards
climate_data = climate_data.filter(col('year') >= 2006)

In [86]:
climate_data.show(2)

+----+-----+-----------+-------------+
|year|month|rainfall_mm|temperature_c|
+----+-----+-----------+-------------+
|2006|  Jan|    22.1909|       26.012|
|2006|  Feb|    22.2449|      26.9355|
+----+-----+-----------+-------------+
only showing top 2 rows



In [87]:
# Calculate the means of rainfall and temperature for 2006-2016
means = climate_data.filter(col('year') <= 2016).select(
    mean('rainfall_mm').alias('mean_rainfall'),
    mean('temperature_c').alias('mean_temperature')
).collect()[0]


In [88]:
from pyspark.sql import Row

# Create rows for missing years (2017-2024) with monthly data, rounded to 2 decimal places
new_rows = []
for year in range(2017, 2025):
    for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
        new_rows.append(Row(
            year=year,
            month=month,
            rainfall_mm=round(means['mean_rainfall'], 2),
            temperature_c=round(means['mean_temperature'], 2)
        ))

In [89]:
# Convert new rows to DataFrame
missing_years_df = spark.createDataFrame(new_rows)

# Combine original and missing data
climate_data = climate_data.union(missing_years_df)

In [90]:
climate_data.show(10)

+----+-----+------------------+------------------+
|year|month|       rainfall_mm|     temperature_c|
+----+-----+------------------+------------------+
|2006|  Jan|22.190900802612305|26.011999130249023|
|2006|  Feb| 22.24489974975586| 26.93549919128418|
|2006|  Mar| 82.78600311279297| 27.03569984436035|
|2006|  Apr| 131.7899932861328|25.886699676513672|
|2006|  May| 83.39550018310547|24.986000061035156|
|2006|  Jun|25.929100036621094|24.407400131225586|
|2006|  Jul| 26.06089973449707| 23.54800033569336|
|2006|  Aug|  44.2327995300293| 24.29490089416504|
|2006|  Sep| 49.32619857788086| 24.12689971923828|
|2006|  Oct| 94.12000274658203|25.804500579833984|
+----+-----+------------------+------------------+
only showing top 10 rows



In [91]:
climate_data.select(min('year')).show()
climate_data.select(max('year')).show()

+---------+
|min(year)|
+---------+
|     2006|
+---------+

+---------+
|max(year)|
+---------+
|     2024|
+---------+



# COMBINED DATA

In [94]:
# Join the tables on 'year' and 'month'
food_combined_data = food_prices2.join(climate_data, on=['year', 'month'], how='left')

In [95]:
# Show the combined table
food_combined_data.show(50)

+----+-----+-------------+-----------+--------------------+------------------+--------------------+----------------+-----------+---------------+------------------+------------------+
|year|month|       region|     county|         market_name|     food_category|           food_item|measurement_unit|local_price|normalized_unit|       rainfall_mm|     temperature_c|
+----+-----+-------------+-----------+--------------------+------------------+--------------------+----------------+-----------+---------------+------------------+------------------+
|2006|  Feb|        Coast|    Mombasa|             Mombasa|cereals and tubers|               Maize|              KG|      16.41|            1.0| 22.24489974975586| 26.93549919128418|
|2006|  Feb|        Coast|    Mombasa|             Mombasa|cereals and tubers|       Maize (white)|           90 KG|     1523.0|           90.0| 22.24489974975586| 26.93549919128418|
|2006|  Feb|        Coast|    Mombasa|             Mombasa|   pulses and nuts|       

In [98]:
from pyspark.sql.functions import round

food_combined_data = food_combined_data.withColumn("rainfall_mm", round(col("rainfall_mm"), 2)).withColumn("temperature_c", round(col("temperature_c"), 2))

In [100]:
food_combined_data.show(2)

+----+-----+------+-------+-----------+------------------+-------------+----------------+-----------+---------------+-----------+-------------+
|year|month|region| county|market_name|     food_category|    food_item|measurement_unit|local_price|normalized_unit|rainfall_mm|temperature_c|
+----+-----+------+-------+-----------+------------------+-------------+----------------+-----------+---------------+-----------+-------------+
|2006|  Jan| Coast|Mombasa|    Mombasa|cereals and tubers|        Maize|              KG|      16.13|            1.0|      22.19|        26.01|
|2006|  Jan| Coast|Mombasa|    Mombasa|cereals and tubers|Maize (white)|           90 KG|     1480.0|           90.0|      22.19|        26.01|
+----+-----+------+-------+-----------+------------------+-------------+----------------+-----------+---------------+-----------+-------------+
only showing top 2 rows



In [99]:
food_combined_data.count()

12824

In [ ]:
data_path = '/content/drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets/clean_data/food_combined_data.csv'

climate_data.write.csv(data_path, header=True, mode='overwrite')

# GIt Version Control Setup

# brc0d3s (dev Branch)

In [ ]:
%cd /content/drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets

/content/drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets


In [ ]:
!git pull origin main

From https://github.com/brc0d3s/Distributed-Food-Price-Prediction-for-Kenyan-Markets
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
!git add .

In [ ]:
!git commit -m "Data Cleaning"

[dev c73c1fd] Data Cleaning
 27 files changed, 14146 insertions(+), 17558 deletions(-)
 rewrite Food_Data_Cleaning.ipynb (98%)
 create mode 100644 clean_data/climate_data.csv/._SUCCESS.crc
 create mode 100644 clean_data/climate_data.csv/.part-00000-dfa16bb0-c078-4d82-8353-fe1ecc67d3df-c000.csv.crc
 create mode 100644 clean_data/climate_data.csv/_SUCCESS
 create mode 100644 clean_data/climate_data.csv/part-00000-dfa16bb0-c078-4d82-8353-fe1ecc67d3df-c000.csv
 create mode 100644 clean_data/food_prices.csv/._SUCCESS.crc
 create mode 100644 clean_data/food_prices.csv/.part-00000-934ad9ce-ad52-4d51-b689-edae1cf98961-c000.csv.crc
 create mode 100644 clean_data/food_prices.csv/_SUCCESS
 create mode 100644 clean_data/food_prices.csv/part-00000-934ad9ce-ad52-4d51-b689-edae1cf98961-c000.csv
 create mode 100644 clean_data/food_production.csv/._SUCCESS.crc
 create mode 100644 clean_data/food_production.csv/.part-00000-fcf5a8ff-eba3-47b8-8833-6f2a192d6dc6-c000.csv.crc
 create mode 100644 clean_data/

In [ ]:
!git push origin dev

Enumerating objects: 23, done.
Counting objects: 100% (23/23), done.
Delta compression using up to 2 threads
Compressing objects: 100% (19/19), done.
Writing objects: 100% (21/21), 247.82 KiB | 1023.00 KiB/s, done.
Total 21 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 1 local object.
To https://github.com/brc0d3s/Distributed-Food-Price-Prediction-for-Kenyan-Markets.git
   a0015fc..c73c1fd  dev -> dev


# barth123 (barth Branch)

In [ ]:
%cd /content/drive/MyDrive/Distributed-Food-Price-Prediction-for-Kenyan-Markets

/content/drive/MyDrive


In [ ]:
!git pull

'Barth ATS standard Resume (1).pdf'	      housing.csv
'Barth ATS standard Resume.pdf'		      housing.gsheet
'barth cv.docx'				      IMG_20250121_172040_147.jpg
'BATHOLOMEN-OGUTU-NYONGESA-REPORT (1).docx'   Lab1.ipynb
'BATHOLOMEN-OGUTU-NYONGESA-REPORT (2).docx'   Relizane_Data.xlsx
 BATHOLOMEN-OGUTU-NYONGESA-REPORT.docx	     'Scan 01 Dec 21 · 03·59·05.pdf'
'Batholomew Nyongesa cv.docx'		     'Transcript-1046075 (2).pdf'
'Batholomew Nyongesa cv.pdf'		     'Transcript-1046075 (3).pdf'
 CoinbaseWalletBackups			      TrustWalletBackup
'Colab Notebooks'			      Untitled0.ipynb
 DMLLabworks.ipynb			     'Untitled document.gdoc'
'Getting started.pdf'			     'Untitled spreadsheet.gsheet'


In [ ]:
!git add .

In [ ]:
!git commit -m "Data Cleaning"

[dev 3056fb5] Data Cleaning
 3 files changed, 1 insertion(+), 1 deletion(-)
 create mode 100644 Abstract/ABSTRACT_GROUP20.docx
 create mode 100644 Abstract/ABSTRACT_GROUP20.pdf


In [ ]:
!git push origin dev

Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 114.97 KiB | 3.48 MiB/s, done.
Total 6 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/brc0d3s/Distributed-Food-Price-Prediction-for-Kenyan-Markets.git
   c73c1fd..3056fb5  dev -> dev
